In [6]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.providers.aer import QasmSimulator
from qiskit.circuit.library import MCMT

import numpy as np
from bitstring import BitArray

# the following function builds a gate (oracle) that transforms |x>|y> to |x>|y - cut(x)>
def energy_oracle(graph_laplacian: list[list[int]], digits: int):

    # setting up the circuit
    qregx = QuantumRegister(len(graph_laplacian),"x")
    qregy = QuantumRegister(digits,"y")
    qc = QuantumCircuit(qregx,qregy)

    # QFT
    for i in range(digits):
        qc.h(qregy[i])

        theta = np.pi/2
        for j in range(i + 1, digits):
            qc.cp(theta, qregy[j], qregy[i])
            theta /= 2

    for i in range(digits // 2):
        qc.swap(qregy[i],qregy[- (i + 1)])

    # Phasers
    for i in range(len(graph_laplacian)):
        theta = - graph_laplacian[i][i] * np.pi
        for j in range(digits):
            qc.cp(theta, qregx[i], qregy[j])
            theta /= 2

        for j in range(i + 1, len(graph_laplacian[i])):
            if graph_laplacian[i][j] != 0:
                theta = - graph_laplacian[i][j] * np.pi * 2
                for k in range(digits):
                    qc.mcp(theta, [qregx[i], qregx[j]], qregy[k])
                    theta /= 2

    # Inverse QFT
    for i in range(digits // 2):
        qc.swap(qregy[i],qregy[- (i + 1)])

    for i in range(digits - 1, - 1, - 1):
        qc.h(qregy[i])

        theta = - np.pi/2
        for j in range(i - 1, - 1, - 1):
            qc.cp(theta, qregy[j], qregy[i])
            theta /= 2

    return qc.to_gate()



# the following function builds the G(alpha, beta) gates for the fixed point search
def grover_bangbang(graph_laplacian: list[list[int]], digits: int, alpha: float, beta: float):
    
    U_f = energy_oracle(graph_laplacian, digits)
    U_f.label = "U_f"
    U_f_inv = U_f.inverse()
    U_f_inv.label = "U_f_inv"

    qregx = QuantumRegister(len(graph_laplacian), "x")
    qregy = QuantumRegister(digits, "y")

    grover_bangbang = QuantumCircuit(qregx,qregy)

    # S_t (beta)
    grover_bangbang.append(U_f, qregx[:] + qregy[:])
    grover_bangbang.p(beta, qregy[0])
    grover_bangbang.append(U_f_inv, qregx[:] + qregy[:])

    # S_s (alpha)
    grover_bangbang.h(qregx)
    grover_bangbang.x(qregx)
    grover_bangbang.mcp(- alpha, qregx[:-1], qregx[-1])
    grover_bangbang.x(qregx)
    grover_bangbang.h(qregx)

    # gateify 
    GBB = grover_bangbang.to_gate()
    GBB.label = "GBB"

    return GBB



# creates the S_L circuit
def grover_fixed_point_circuit(graph_laplacian: list[list[int]], digits: int, l: int, delta: float):

    alpha = []

    gamma = 1 / np.cosh(np.arccosh(1 / delta) / (2 * l + 1))
    gamma = np.sqrt(1 - gamma * gamma)

    for j in range(1, l + 1):        
        alpha.append(2 * np.arctan(1 / (gamma * np.tan(2 * np.pi * j / (2 * l + 1)))))

    qregx = QuantumRegister(len(graph_laplacian), "x")
    qregy = QuantumRegister(digits, "y")

    qc = QuantumCircuit(qregx,qregy)

    qc.h(qregx)

    for j in range(l):
        G_j = grover_bangbang(graph_laplacian, digits, alpha[j], - alpha[l - 1 - j])
        G_j.label = "G(alpha_" + str(j + 1) + ", beta_" + str(j + 1) + ")"
        qc.append(G_j, qregx[:] + qregy[:])

    GFPC = qc.to_gate()
    GFPC.label = "GFPC"

    return GFPC


# This runs the actual search
def grover_fixed_point_search(graph_laplacian: list[list[int]], digits: int, y: int, delta: float, l: int, shot_number: int):

    # bitifying y
    y_bits = []
    Y = y
    while Y != 0 or len(y_bits) < digits:
        y_bits.append(Y&1)
        Y >>= 1

    y_bits = y_bits[::-1]

    GFPC = grover_fixed_point_circuit(graph_laplacian, digits, l, delta)

    # one quantum/classical register pair for each vertex
    qregx = QuantumRegister(len(graph_laplacian), "x")
    cregx = ClassicalRegister(len(graph_laplacian), "cl-x")
    # one quantum/classical register pair for each digit
    qregy = QuantumRegister(digits, "y")
    cregy = ClassicalRegister(digits, "cl-y")

    qcircuit = QuantumCircuit(qregx,qregy,cregx,cregy)

    for i in range(len(qregx)):
        qcircuit.initialize([1, 0], qregx[i])

    for i in range(len(qregy)):
        qcircuit.initialize([1 - y_bits[i], y_bits[i]], qregy[i])

    qcircuit.append(GFPC, qregx[:] + qregy[:])

    # add one more energy oracle to see the cut values
    U_f = energy_oracle(graph_laplacian, digits)
    U_f.label = "energy"
    qcircuit.append(U_f, qregx[:] + qregy[:])

    # measure
    qcircuit.measure(qregx, cregx)
    qcircuit.measure(qregy, cregy)


    #print(qcircuit.draw())

    simulator = QasmSimulator()
    compiled_qcircuit = transpile(qcircuit, simulator)
    job = simulator.run(compiled_qcircuit, shots=shot_number)
    result = job.result()

    return result.get_counts(compiled_qcircuit)


# TEST

#graph_laplacian = [[3, - 1, - 1, - 1], [- 1, 1, 0, 0], [- 1, 0, 1, 0], [- 1, 0, 0, 1]]
graph_laplacian = [[4, - 1, - 1, - 1, - 1], [- 1, 3, - 1, -1, 0], [- 1, - 1, 3, - 1, 0], [- 1, - 1, - 1, 4, - 1], [- 1, 0, 0, - 1, 2]]
digits = 4
y = 4
delta = np.sqrt(0.1)
M_over_N = 14/pow(2, len(graph_laplacian))
shot_number = 1000

# from the input we compute the follwing
l = int(np.ceil(np.arccosh(1 / delta) / np.arccosh(1 / np.sqrt(1 - M_over_N * M_over_N))))# // 2
P_L = 1 - pow(delta * np.cos((2 * l + 1) * np.arccos(np.cosh(np.arccosh(1 / delta) / (2 * l + 1)) * np.sqrt(1 - M_over_N * M_over_N))), 2)

# results

counts = grover_fixed_point_search(graph_laplacian, digits, y, delta, l, shot_number)

adjusted_counts = []

for s in counts:
    adjusted_counts.append([s[::-1], 100 * counts[s] / shot_number])

adjusted_counts.sort()

print()
print("graph laplacian")
print("---------------")
for x in graph_laplacian:
    print(x)
print()
print("y =",y)
print("l =",l)
print("N/M =",M_over_N)
print("delta =",delta)
print()
prob = 0
for s in adjusted_counts:
    b = s[0][-digits:]
    if b[0] == "1":
        prob += s[1]
    cut = BitArray(bin=b).int
    cut = y - cut
    print("configuration =",s[0][:len(graph_laplacian)],"result =",b," cut =",cut," frequency =",s[1],"%")
print()
print("Probability of success =",prob,"%")
print("(P_L =",P_L * 100,"%)")
print()


graph laplacian
---------------
[4, -1, -1, -1, -1]
[-1, 3, -1, -1, 0]
[-1, -1, 3, -1, 0]
[-1, -1, -1, 4, -1]
[-1, 0, 0, -1, 2]

y = 4
l = 4
N/M = 0.4375
delta = 0.31622776601683794

configuration = 00000 result = 0100  cut = 0  frequency = 1.1 %
configuration = 00001 result = 0010  cut = 2  frequency = 0.4 %
configuration = 00010 result = 0000  cut = 4  frequency = 1.0 %
configuration = 00011 result = 0000  cut = 4  frequency = 0.7 %
configuration = 00100 result = 0001  cut = 3  frequency = 0.6 %
configuration = 00101 result = 1111  cut = 5  frequency = 6.8 %
configuration = 00110 result = 1111  cut = 5  frequency = 7.0 %
configuration = 00111 result = 1111  cut = 5  frequency = 6.4 %
configuration = 01000 result = 0001  cut = 3  frequency = 0.3 %
configuration = 01001 result = 1111  cut = 5  frequency = 6.4 %
configuration = 01010 result = 1111  cut = 5  frequency = 6.3 %
configuration = 01011 result = 1111  cut = 5  frequency = 4.9 %
configuration = 01100 result = 0000  cut = 4  fr